# Details of the tool scripts

## Pre-clustering
This step aims to identify groups of similar sequences to form initial clonal groups that will be refined later
+ Creat the GTG input file

In [17]:
%run format_labeling_imgt.py -s I1_IMGT/1_Summary.txt -g I1_IMGT/2_IMGT-gapped-nt-sequences.txt -o I1_IMGT_seq_Fo.txt

   The format_labeling_imgt's output file, saved as `I1_IMGT_seq_Fo_V_CDR3_Jseq.txt`, contains the followong colums 

<pre>
sequence_name  productivity_statut   IGHV_gene_and_allele    IGHJ_gene_and_allele    CDR3_in_AA   Junction 

Example :
S1	productive	IGHV3-74*01	IGHJ4*02	CARDQGSADTGVGTAVPFDSW	ctttgactcctggggccagggaaccctggtcaccgtctcctcag
S2	productive	IGHV3-74*01	IGHJ4*02	CVRDQGSADTGVGTAVPFDSW	ctttgactcatggggccagggaaccctggtcaccgtctcctcag
S3	productive	IGHV3-74*01	IGHJ4*02	CVRDQGSADTGVGTAVPFDSW	ctttgactcatggggccagggaaccctggtcaccgtctcctcag
S4	productive	IGHV3-74*01	IGHJ4*02	CARDQGSADTGVGTAVPFDSW	ctttgactcctggggccagggaaccctggtcaccgtctcctcag
</pre>

+ Creat the initial clusters by grouping sequences with the same IGHV and IGHJ genes, and same CDR3 sequence length. Next, we separate sequences with less than 70% of CDR3 identity.

In [29]:
import sys
!{sys.executable} -m pip install levenshtein
%run initial_clustering.py -i I1_IMGT_seq_Fo_V_CDR3_Jseq.txt -o I1_IMGT -s 0.7
%run format_clustering_output.py -i I1_IMGT_sameVJ_noallele_CDR3_0.7.txt -o I1_IMGT_initial_clusters_Fo.txt

Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/lib/python3/dist-packages/pip/commands/install.py", line 353, in run
    wb.build(autobuilding=True)
  File "/usr/lib/python3/dist-packages/pip/wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 554, in _prepare_file
    require_hashes
  File "/usr/lib/python3/dist-packages/pip/req/req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 465, in find_requirement
    all_candidates = self.find_all_candidates(req.name)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 423, in fin

ModuleNotFoundError: No module named 'Levenshtein'

In [18]:
python  initial_clustering.py -i ${name}_seq_Fo_V_CDR3_Jseq.txt -o ${name} -s 0.7
python format_clustering_output.py -i ${name}_sameVJ_noallele_CDR3_0.7.txt -o ${name}_initial_clusters_Fo.txt
python refinement.py -f ${name}_seq_Fo_V_CDR3_Jseq.txt -c ${name}_initial_clusters_Fo.txt
python format_clustering_output.py -i ${name}_seq_Fo_V_CDR3_Jseq_clone_V_CDR3_J.txt -o ${name}_final_clusters_Fo.txt
python two_level_clonal_info.py -f ${name}_seq_Fo_V_CDR3_Jseq.txt -c ${name}_final_clusters_Fo.txt -n ${name}
python Cluster_distribution_plot.py -n ${name} -c ${name}_final_clusters_Fo.txt

SyntaxError: invalid syntax (<ipython-input-18-8879756f531e>, line 1)

## Clustering refining
+ we iteratively refine initial clonal groups until it is not possible to minimize intraclonal distances or maximize interclonal distances.

In [30]:
%run format_clustering_output.py -i I1_IMGT_seq_Fo_V_CDR3_Jseq_clone_V_CDR3_J.txt -o I1_IMGT_final_clusters_Fo.txt
%run two_level_clonal_info.py -f I1_IMGT_seq_Fo_V_CDR3_Jseq.txt -c I1_IMGT_final_clusters_Fo.txt -n I1_IMGT

FileNotFoundError: [Errno 2] No such file or directory: 'I1_IMGT_seq_Fo_V_CDR3_Jseq_clone_V_CDR3_J.txt'

FileNotFoundError: [Errno 2] No such file or directory: 'I1_IMGT_final_clusters_Fo.txt'

Two main output files are :

1) The `I1_IMGT_final_clusters_seq_info.txt` containes :

<pre>
Clone_clonotype    sequence_name  productivity_statut   IGHV_gene_and_allele    IGHJ_gene_and_allele    CDR3_in_AA   Junction

Example :
0_0	S1	productive	IGHV3-74*01	IGHJ4*02	CARDQGSADTGVGTAVPFDSW	ctttgactcctggggccagggaaccctggtcaccgtctcctcag
0_0	S4	productive	IGHV3-74*01	IGHJ4*02	CARDQGSADTGVGTAVPFDSW	ctttgactcctggggccagggaaccctggtcaccgtctcctcag
1_0	S9	productive	IGHV3-74*01	IGHJ4*02	CARDQGSADTGVGTAVPFDSW	ctttgactcctggggccagggaaccctggtcaccgtctcctcag
1_1	S35	productive	IGHV3-74*01	IGHJ4*02	CARDQGSADTGVGTAVPFDSW	ctttgactcctggggccagggaaccctggtcaccgtctcctcag
</pre>

2) The `I1_IMGT_final_clusters_Fo.txt` contains

<pre>
clone    Seq_1  Seq_2 

Example : 
111	S2181 S19881 S22000 S31862
112	S2384
</pre>